**Copyright 2023 Google LLC.**

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2023 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img align="left" width="150" src="https://services.google.com/fh/files/misc/ml_toast_logo.png" alt="ml_toast_logo" /><br><br>

# 🍞 ML-ToAST: **M**ulti**l**ingual **To**pic Clustering of **A**ds-triggering **S**earch **T**erms

**Disclaimer: This is not an official Google product.**

**🍞 ML-ToAST** is an open-source tool that helps users cluster multilingual search terms captured from different time windows into semantically relevant topics. It helps advertisers / marketers surface insights related to changing consumer interest in a configurable, user-friendly, and privacy-safe manner.

More information available at [github.com/google/ml_toast](https://github.com/google/ml_toast).

## Prerequisites

### Which search terms to use?

We recommend extracting the Google Ads [Search Terms report](https://support.google.com/google-ads/answer/2472708) for the following periods:
 * **Last 30 days** (e.g. Nov 1 - Nov 30): it generally makes sense to look at the most recent search terms that triggered your ads.
 * **Previous 30/31 days** (e.g. Oct 1 - Oct 31): this helps provide information on those search terms that constitute your core business over those that are recently trending.
 * **Last 30 days last year** (e.g. Nov 1 - Nov 30 of the previous year): to account for seasonality effects (e.g. holiday season).

We also recommend restricting the extracted search terms to a subset of *related* campaigns (e.g. all campaigns for a specific *product line* or *operating domain*) rather than all campaigns in your account. This allows the model to better capture how the search terms relate to one another, and therefore, extract more meaningful topics.

The report can be downloaded from the Google Ads UI in CSV format and imported into a Google Sheets spreadsheet or a Google Cloud BigQuery table, to be used below for input/output.

*Note: if you have multiple accounts operating under the same product line or domain, you can extract search terms from those accounts as well and group them all into the same Google Sheets spreadsheet / Google Cloud BigQuery table.*

## Get Started

In [ ]:
#@title Authenticate your user for this colab session
import logging
from google.colab import auth

auth.authenticate_user()
logging.getLogger().setLevel(logging.INFO)

In [ ]:
#@title Install dependencies
!pip install tensorflow-text hdbscan umap-learn

## Input and Preprocessing - Google Sheets


In [ ]:
#@title Configurable params { run: 'auto' }

#@markdown Enter your spreadsheet ID:
spreadsheet_id = "id-goes-here" #@param {type:"string"}

#@markdown Enter the main worksheet name (which should usually contain the search terms from the last month):
input_sheet_name = "colab-input-main" #@param {type:"string"}

#@markdown Enter the prefix for any additional worksheets you would also like to analyze (e.g. search terms from the previous month, previous year, etc.):
additional_sheets_prefix = "colab-input-lookback-" #@param {type:"string"}

#@markdown Enter the name of the column that contains search terms. This value should be the same across all worksheets (defaults to the column name in the Google Ads Search Terms report):
search_terms_column = "Search term" #@param {type:"string"}

#@markdown <hr>Filtering settings

#@markdown ***Check*** the checkbox to filter on new terms (i.e. compare search terms from the aforementioned lookback worksheets and keep only what is *new*) and ***uncheck*** to analyze search terms from the main worksheet only.
filter_new_terms = True #@param {type:"boolean"}

#@markdown Enter the name of a column that contains a metric you would like to use for filtering and/or sorting (e.g. impressions):
filter_metric_column = "Impr." #@param {type:"string"}

#@markdown Use this value to indicate the maximum allowed threshold for your chosen *metric* (e.g. 1000 will filter all search terms with *metric* > 1000).<br>Set to *-1* to skip *metric* filtering.
filter_metric_max_threshold = -1 #@param {type:"integer"}

#@markdown Select the desired logical grouping (AND/OR) of the *new* and *metric* filters:
filters_grouping = "AND" #@param ["AND", "OR"]

# Google Ads specific params

known_report_metrics = ['Clicks', 'Impr.', 'Cost']
all_report_metrics = (
    known_report_metrics if filter_metric_column in known_report_metrics
    else [filter_metric_column])

# Validation rules
if not spreadsheet_id or not input_sheet_name or not search_terms_column:
  raise ValueError(
      'Invalid input! Please make sure at least '
      '"spreadsheet_id", "input_sheet_name" and "search_terms_column" '
      'are provided.')


In [ ]:
#@title Fetch data from the input spreadsheet
#@markdown The first row in each worksheet will be considered the **column headers** row.
import pandas as pd
import gspread
from google.auth import default

creds, _ = default()
sheets_client = gspread.authorize(creds)
spreadsheet = sheets_client.open_by_key(spreadsheet_id)

input_values = spreadsheet.worksheet(input_sheet_name).get_all_values()
additional_sheets_values = []

if filter_new_terms and additional_sheets_prefix:
  for sheet in spreadsheet.worksheets():
    if sheet.title.startswith(additional_sheets_prefix):
      additional_sheets_values.append(sheet.col_values(1))

input_data = pd.DataFrame(input_values[1:], columns=input_values[0])

for report_metric in all_report_metrics:
  if report_metric in input_data.columns:
    input_data[report_metric] = pd.to_numeric(
        input_data[report_metric].str.replace(',', ''))

print(
    f'Worksheet: {input_sheet_name}\nNumber of rows: {len(input_data)}\n'
    'First 5 rows:')
input_data.head()

In [ ]:
#@title Extract search terms and apply the defined filters

def add_filter(existing_filter, new_filter):
  if filters_grouping == 'AND':
    return existing_filter & new_filter
  return existing_filter | new_filter

additional_data = [
    pd.DataFrame(sheet_values[1:], columns=[sheet_values[0]])
    for sheet_values in additional_sheets_values]

data_unfiltered = input_data.copy()

series_filter = (filters_grouping == 'AND')
applied_filters = []

if additional_data:
  data_unfiltered = (
      data_unfiltered.merge(pd.concat(additional_data).drop_duplicates(),
                    on=search_terms_column,
                    how='left',
                    indicator=True))
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
          data_unfiltered['_merge'] == 'left_only'))
  applied_filters.append('filter_new_terms')

if filter_metric_column and filter_metric_max_threshold > 0:
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
      data_unfiltered[filter_metric_column] <= filter_metric_max_threshold))
  applied_filters.append(
      f'filter_metric_max_threshold < {filter_metric_max_threshold}')

main_input_data = (
    data_unfiltered[series_filter] if applied_filters else data_unfiltered)

if '_merge' in main_input_data.columns:
  main_input_data = main_input_data.drop(columns='_merge')

if filter_metric_column in main_input_data.columns:
  main_input_data = main_input_data.sort_values(
      by=filter_metric_column, ascending=False)

print('\n'.join([
    f'Filtered data - total number of rows: {len(main_input_data)}',
    f'Filters applied: {applied_filters}',
    (
        f"Filters logical grouping: '{filters_grouping}'"
        if len(applied_filters) > 1 else ''),
    'First 5 rows:']))
main_input_data.head()

## Input and Preprocessing - Google Cloud BigQuery


In [ ]:
#@title Configurable params { run: 'auto' }

#@markdown Enter your Google Cloud Project ID:
gcp_project_id = "id-goes-here" #@param {type:"string"}

#@markdown Enter the SQL query you would like to execute to pull data from BigQuery
#@markdown (*Expand this cell to view a sample query for the [Google Ads BigQuery Data Transfer](https://cloud.google.com/bigquery/docs/google-ads-transfer)*):
bq_input_query = "SELECT * FROM `dataset.table` WHERE column = \"value\" ORDER BY column" #@param {type:"string"}

# Google Ads BigQuery Data Transfer sample query
gcp_dataset_name = '<dataset>'
gcp_ads_transfer_tablename_suffix = '_<suffix>'
bg_input_query_sample = f"""
SELECT
  S.search_term_view_search_term AS `Search term`, 
  S.segments_search_term_match_type AS `Match type`,
  S.search_term_view_status AS `Added or Excluded`, 
  C.campaign_name AS `Campaign`, 
  C.campaign_advertising_channel_type `Campaign type`, 
  C.campaign_bidding_strategy_type AS `Bid strategy type`, 
  A.ad_group_name AS `Ad group`,
  SUM(S.metrics_impressions) AS `Impr`, 
  SUM(S.metrics_clicks) AS `Clicks`, 
  SUM(S.metrics_cost_micros) AS `Cost`,
FROM
  `{gcp_project_id}.{gcp_dataset_name}.ads_SearchQueryStats{gcp_ads_transfer_tablename_suffix}` AS S
INNER JOIN 
  `{gcp_project_id}.{gcp_dataset_name}.ads_Campaign{gcp_ads_transfer_tablename_suffix}` AS C USING (campaign_id)
INNER JOIN 
  `{gcp_project_id}.{gcp_dataset_name}.ads_AdGroup{gcp_ads_transfer_tablename_suffix}` AS A USING (ad_group_id)
GROUP BY
  S.search_term_view_search_term, 
  S.segments_search_term_match_type,
  S.search_term_view_status, 
  C.campaign_name, 
  C.campaign_advertising_channel_type, 
  C.campaign_bidding_strategy_type, 
  A.ad_group_name
ORDER BY `Impr` DESC, `Search term`;
"""

#@markdown Enter the name of the column that contains search terms:
search_terms_column = "Search term" #@param {type:"string"}

#@markdown Enter the name of the BigQuery dataset where output should be stored:
bq_output_dataset = "ml_toast_output" #@param {type:"string"}

# Validation rules
if not gcp_project_id or not bq_input_query:
  raise ValueError(
      'Invalid input! Please make sure a valid GCP '
      '"project_id" and "bq_query" are provided.')

if not bq_output_dataset:
  print(
      'WARNING - "bq_output_dataset" is not set. '
      'Writing output to BigQuery will fail!'
  )

In [ ]:
#@title Fetch input data from GCP BigQuery
%%bigquery main_input_data --project $gcp_project_id
$bq_input_query

## Input and Preprocessing - Common Settings

Run these cells regardless of the input type


In [ ]:
#@title Configurable params { run: 'auto' }

#@markdown Enter the name of the column that represents the **match type** for each search term (defaults to the column name in the Google Ads Search Terms report):
match_type_column = "Match type" #@param {type:"string"}

#@markdown Enter the *match type* value that corresponds to Broad Match (defaults to the column name in the Google Ads Search Terms report):
match_type_broad = "Broad match" #@param {type:"string"}

#@markdown <hr>Advanced settings

#@markdown Enter a comma-separated list of *stop words* which should be excluded from all generated topics:
stop_words = "stop1, stop2" #@param {type:"string"}

if stop_words:
  stop_words = stop_words.replace(', ', ',').split(',')
else:
  stop_words = None

#@markdown ***Check*** the checkbox to perform hyperparameter tuning for UMAP + HDBSCAN (increases processing time by a factor of ~3).<br>
#@markdown Despite the time factor, we **highly** recommend using this to provide the optimal results for the given input.<br>
#@markdown ***Uncheck*** to use the default clustering parameters.
hyperparameter_tuning = False #@param {type:"boolean"}

# Debugging params
# Calculates performance metrics for the output topics, which will be done in
# the provided LookerStudio dashboard (set to True if not using the dashboard)
output_topic_metrics = False

In [ ]:
#@title Display the first 5 rows of the fetched data

if main_input_data.empty:
  raise ValueError(
      'No data was fetched. Please run all cells in either the "Google Sheets" '
      'or "Google Cloud BigQuery" Input and Preprocessing section first.')

main_input_data.head()


In [ ]:
#@title Extract 'Broad Match' and 'non Broad Match' terms from the fetched data
broad_match_groups = {}

if match_type_column in main_input_data.columns:
  broad_match_terms = main_input_data.copy()
  broad_match_terms = broad_match_terms[
      main_input_data[match_type_column] == match_type_broad]

  non_broad_match_terms = main_input_data.copy()
  non_broad_match_terms = non_broad_match_terms[
      main_input_data[match_type_column] != match_type_broad]

  if not broad_match_terms.empty:
    broad_match_groups['broad_match_terms'] = broad_match_terms

  if not non_broad_match_terms.empty:
    broad_match_groups['non_broad_match_terms'] = non_broad_match_terms

  print(
      'Extracted:\n'
      f' - All terms where "{match_type_column}" is "{match_type_broad}" '
      f'from the filtered data. Number of rows: {len(broad_match_terms)}')
  print(
      'Extracted:\n'
      f' - All terms where "{match_type_column}" is NOT "{match_type_broad}" '
      f'from the filtered data. Number of rows: {len(non_broad_match_terms)}')
else:
  print(
      f'No column "{match_type_column}" found in the input data. '
      'Skipping extraction of Broad Match terms.')

## Topic Clustering

In [ ]:
#@title Import the topic clustering library
!echo "Restoring working directory to root..."
%cd /content
!rm -rf ml_toast && git clone https://github.com/google/ml_toast.git
!echo "Changing working directory to ml_toast..."
%cd ml_toast

from ml_toast import topic_clustering as topic_clustering_lib

In [ ]:
#@title Use the library to determine topics for all input groups
terms_input_data_dict = {'input_terms': main_input_data} | broad_match_groups

for key, terms_input_data in terms_input_data_dict.items():
  topic_clustering = topic_clustering_lib.TopicClustering(
      data_id=key,
      input_col=search_terms_column,
      stop_words=stop_words,
      do_hdbscan_hyperopt=hyperparameter_tuning)
  topics_kmeans, topics_hdbscan = topic_clustering.determine_topics(
      terms_input_data)
  terms_input_data['Topic'] = topics_kmeans
  terms_input_data['Additional Topics'] = topics_hdbscan

## Output

Run only one of the cells below, depending on the origin of your input.

In [ ]:
#@title Write results back to the input spreadsheet
#@markdown New worksheets with the prefix **colab-** will be appended to the spreadsheet,
#@markdown or overwritten if they already exist.
from gspread_dataframe import set_with_dataframe

for key, terms_input_data in terms_input_data_dict.items():
  try:
    output_sheet = spreadsheet.worksheet(f'colab-{key}-output')
    output_sheet.clear()
  except gspread.exceptions.WorksheetNotFound:
    output_sheet = spreadsheet.add_worksheet(
        f'colab-{key}-output',
        rows=len(terms_input_data),
        cols=len(terms_input_data.columns))
  set_with_dataframe(
      output_sheet, terms_input_data, include_column_header=True)

  if output_topic_metrics:
    for topics_type in ['Topic', 'Additional Topics']:
      cluster_metrics = pd.DataFrame()
      cluster_metrics['Topic'] = terms_input_data[topics_type]
      for report_metric in all_report_metrics:
        if report_metric in terms_input_data.columns:
          cluster_metrics[report_metric] = terms_input_data[report_metric]

      cluster_metrics = cluster_metrics.groupby(by='Topic', sort=False).agg(
          ['mean', 'median', 'min', 'max', 'std', 'var'])
      cluster_metrics.insert(loc=0, column='Topic', value=cluster_metrics.index)
      cluster_metrics.insert(
          loc=1,
          column='Count',
          value=terms_input_data.groupby(by=topics_type, sort=False).count()[
              search_terms_column])
      cluster_metrics = cluster_metrics.sort_values(by='Count', ascending=False)
      cluster_metrics_key = (
          f"{key}_{topics_type.lower().replace(' ', '_')}")

      try:
        metrics_sheet = spreadsheet.worksheet(
            f'colab-{cluster_metrics_key}-metrics')
        metrics_sheet.clear()
      except gspread.exceptions.WorksheetNotFound:
        metrics_sheet = spreadsheet.add_worksheet(
            f'colab-{cluster_metrics_key}-metrics',
            rows=len(cluster_metrics),
            cols=len(cluster_metrics.columns))
      set_with_dataframe(
          metrics_sheet, cluster_metrics, include_column_header=True)

In [ ]:
#@title Write results to new GCP BigQuery tables
#@markdown **Optional**: Modify the `destination_table` and `if_exists` parameters below as desired (e.g. adding a prefix to the name of the created tables).

for key, terms_input_data in terms_input_data_dict.items():
  terms_input_data.to_gbq(
      destination_table=f'{bq_output_dataset}.{key}', # `dataset.tablename`
      project_id=gcp_project_id,
      if_exists='replace',  # fail/replace/append
  )


### Visualize the generated topics in LookerStudio

ML-ToAST provides a template LookerStudio dashboard to help you visualize the generated topics and quickly surface insights.

Use [this link](https://lookerstudio.google.com/c/u/0/reporting/8bc2240e-a919-4916-9c7f-daf72f75bf42/preview) to create a copy of the dashboard and get started! All you would need to do is map the data sources used by the dashboard to the spreadsheet / BigQuery tables used for the input and output above. The dashboard contains both types of data sources, so please delete the ones that are irrelevant / unused.

### How was the sample data generated?

All data in the template LookerStudio dashboard was generated using the `Faker` library as shown by the cells below.

In [ ]:
#@title Install dependencies
!pip install Faker

In [ ]:
#@title Generate sample data with Faker
#@markdown The generated data format resembles that of a typical Google Ads Search Terms report.
import random

import pandas as pd
from faker import Faker

GADS_MATCH_TYPES = [
    'Broad match',
    'Exact match',
    'Exact match (close variant)',
    'Phrase match',
    'Phrase match (close variant)',
]

fake = Faker(['en'])
rows = [['Search term', 'Match type', 'Campaign', 'Ad group', 'Impr.', 'Clicks']]

for i in range (1, 50000):
  row = [
      fake.text(max_nb_chars=50).replace('.', ''),
      fake.random_element(elements=GADS_MATCH_TYPES),
      f'Campaign {random.randint(1, 10)}',
      f'Ad group {random.randint(1, 10)}',
      random.randint(1, 10000),
      random.randint(1, 10000),
    ]
  rows.append(row)

generated_terms = pd.DataFrame(rows[1:], columns=rows[0])
generated_terms.head()

In [ ]:
#@title Output to csv
#@markdown The file will be saved in the default 'home' folder on Colab (`/content/`) and can be downloaded from there.
generated_terms.to_csv('/content/faker_sample.csv', index=False)